In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# User Input

In [ ]:
user_content = "What should I do today?"

# Judgement

In [ ]:
judge_model_id = "microsoft/Phi-3.5-mini-instruct"

# A model to judge if there is relevant information in the given text
judge_model = AutoModelForCausalLM.from_pretrained( 
    judge_model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

judge_model_tokenizer = AutoTokenizer.from_pretrained(judge_model_id)

In [ ]:
# Pipeline for the judgement model
judge_pipe = pipeline(
    "text-generation",
    model=judge_model,
    tokenizer=judge_model_tokenizer,
)

judge_generation_args = {
    "max_new_tokens": 1,
    "return_full_text": False,
    "temperature": 0.1,
    "do_sample": True,
}

In [ ]:
judge_system_content = """
    Your duty is to assist the personal companion. You have to decide if the personal companion will benefit by remebering the facts from the given text.
    The personal companion is expected to remember everything important to an individual. The personal companion is a best friend who takes care of the User.
    The personal companion is a companion for old individuals.
    The personal companion is expected to remember family and related matters, milestones in life, health issues, medication schedules.

    If you think the text contains relevant information, you have to respond with 'TRUE'.
    If there is no relevant information in the text, return 'FALSE'.
    Do NOT say anything else. You have to respond ONLY with either 'TRUE' or 'FALSE'.
    
    You are the Assistant in the given examples.
    
    Examples:
    
    User: Hey, my name is Adam.
    Assistant: TRUE
    
    User: I love my brother John.
    Assistant: TRUE
    
    User: The sky is blue today.
    Assistant: FALSE
"""

In [ ]:
judge_messages = [ 
    {"role": "system", "content": judge_system_content},
    {"role": "user", "content": user_content},
]

In [ ]:
output = judge_pipe(judge_messages, **judge_generation_args)
judgement = output[0]['generated_text']
print(judgement)

# Clear CUDA cache

In [ ]:
# Emptying cuda
torch.cuda.empty_cache()
del judge_model
del judge_model_tokenizer
del judge_pipe

# Memory Extraction

In [ ]:
extract_model_id = "Qwen/Qwen2.5-7B-Instruct"

# Extraction model to extract the relevant information
extract_model = AutoModelForCausalLM.from_pretrained( 
    extract_model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

extract_model_tokenizer = AutoTokenizer.from_pretrained(extract_model_id)

In [ ]:
# Pipeline for the extraction model
extract_pipe = pipeline(
    "text-generation",
    model=extract_model,
    tokenizer=extract_model_tokenizer,
)

extract_generation_args = {
    "max_new_tokens": 100,
    "return_full_text": False,
    "temperature": 0.5,
    "do_sample": True,
}

In [ ]:
extract_system_content = """
    Your duty is to assist the personal companion. You have to extract the facts from the given text that will benefit hte personal companion.
    The personal companion is expected to remember everything important to an individual. The personal companion is a best friend who takes care of the User.
    The personal companion is a companion for old individuals.
    The personal companion is expected to remember family and related matters, milestones in life, health issues, medication schedules.
    
    You have to extract the relevant information from the given text.
    
    You are the Assistant in the given examples.
    
    Examples:
    
    User: Hey, my name is Adam.
    Assistant: Name is Adam.
    
    User: I love my brother John.
    Assistant: Brother named John.
    
    User: The sky is blue today.
    Assistant: 
    
    User: I love my wife Emily and we just came from Coldplay concert. I love them.
    Assistant: Wife named Emily. Likes Coldplay.
"""

In [ ]:
extract_messages = [ 
    {"role": "system", "content": extract_system_content},
    {"role": "user", "content": user_content},
]

In [ ]:
if judgement == ' TRUE':
    output = extract_pipe(extract_messages, **extract_generation_args)
    memory = output[0]['generated_text']
    print(memory)